# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [3]:
project_root = get_project_root() 
billing_id = 'trabalho-pratico-ml'

Para fazer login:
``` shell
gcloud auth application-default login
```

In [ ]:
%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100

# %sql bigquery://genuine-orb-470122-v0
%sql bigquery://trabalho-pratico-ml

E0000 00:00:1763336584.810462  168817 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Connecting to 'bigquery://trabalho-pratico-ml'

In [5]:
%%sql
SELECT * FROM `basedosdados.br_ms_sinan.dicionario` limit 10

Running query in 'bigquery://trabalho-pratico-ml'

10 rows affected.

,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,microdados_dengue,escolaridade_paciente,1,(1),1ª a 4ª série incompleta do EF
1,microdados_dengue,escolaridade_paciente,2,(1),4ª série completa do EF ( antigo 1° grau)
2,microdados_dengue,escolaridade_paciente,3,(1),5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)
3,microdados_dengue,raca_cor_paciente,3,(1),Amarela
4,microdados_dengue,idade_paciente,4,(1),Ano
5,microdados_dengue,raca_cor_paciente,1,(1),Branca
6,microdados_dengue,criterio_confirmacao,2,(1),Clínico Epidemiológico
7,microdados_dengue,evolucao_caso,1,(1),Cura
8,microdados_dengue,idade_paciente,2,(1),Dia
9,microdados_dengue,escolaridade_paciente,8,(1),Educação superior completa


### Testing

In [6]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

Running query in 'bigquery://trabalho-pratico-ml'

26 rows affected.

,ano,qt
0,2000,172855
1,2001,488590
2,2002,897093
3,2003,416609
4,2004,136867
5,2005,261501
6,2006,411022
7,2007,717097
8,2008,919324
9,2009,600658


# bi_query

### low_risk

In [15]:
%%sql df <<
SELECT
    sigla_uf_residencia,
    idade_paciente,
    sexo_paciente,
    raca_cor_paciente,
    gestante_paciente,
    possui_doenca_autoimune,
    possui_diabetes,
    possui_doencas_hematologicas,
    possui_hepatopatias,
    possui_doenca_renal,
    possui_hipertensao,
    possui_doenca_acido_peptica,
    apresenta_febre,
    apresenta_cefaleia,
    apresenta_exantema,
    apresenta_dor_costas,
    apresenta_mialgia,
    apresenta_vomito,
    apresenta_conjutivite,
    apresenta_dor_retroorbital,
    apresenta_artralgia,
    apresenta_artrite,
    apresenta_leucopenia,
    apresenta_petequias,
    prova_laco,
    classificacao_final,
    evolucao_caso,
    DATE_DIFF(data_notificacao, data_primeiros_sintomas, DAY) AS dias_sintomas_notificacao
FROM
    `basedosdados.br_ms_sinan.microdados_dengue`
WHERE
    classificacao_final = '10'
    AND evolucao_caso != '2'
    AND evolucao_caso IS NOT NULL
    AND data_primeiros_sintomas IS NOT NULL
    AND data_notificacao IS NOT NULL
    AND ano = 2021

Running query in 'bigquery://trabalho-pratico-ml'

423173 rows affected.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423173 entries, 0 to 423172
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   sigla_uf_residencia           423173 non-null  object
 1   idade_paciente                423173 non-null  object
 2   sexo_paciente                 423172 non-null  object
 3   raca_cor_paciente             423169 non-null  object
 4   gestante_paciente             423068 non-null  object
 5   possui_doenca_autoimune       423168 non-null  object
 6   possui_diabetes               423168 non-null  object
 7   possui_doencas_hematologicas  423168 non-null  object
 8   possui_hepatopatias           423168 non-null  object
 9   possui_doenca_renal           423168 non-null  object
 10  possui_hipertensao            423168 non-null  object
 11  possui_doenca_acido_peptica   423168 non-null  object
 12  apresenta_febre               423168 non-null  object
 13 

In [17]:
save_parquet_in_chunks(df, "data/2_silver/2021_low_risk")

Salvando arquivos Parquet:   0%|          | 0/423173 [00:00<?, ?linhas/s]

✔️ 2021_low_risk_part_1.parquet salvo com 2.00 MB (423173 linhas)
✅ Todos os arquivos salvos com sucesso.


## alarm + severe

In [18]:
%%sql df <<
SELECT
    sigla_uf_residencia,
    idade_paciente,
    sexo_paciente,
    raca_cor_paciente,
    gestante_paciente,
    possui_doenca_autoimune,
    possui_diabetes,
    possui_doencas_hematologicas,
    possui_hepatopatias,
    possui_doenca_renal,
    possui_hipertensao,
    possui_doenca_acido_peptica,
    apresenta_febre,
    apresenta_cefaleia,
    apresenta_exantema,
    apresenta_dor_costas,
    apresenta_mialgia,
    apresenta_vomito,
    apresenta_conjutivite,
    apresenta_dor_retroorbital,
    apresenta_artralgia,
    apresenta_artrite,
    apresenta_leucopenia,
    apresenta_petequias,
    prova_laco,
    classificacao_final,
    evolucao_caso,
    DATE_DIFF(data_notificacao, data_primeiros_sintomas, DAY) AS dias_sintomas_notificacao
FROM
    `basedosdados.br_ms_sinan.microdados_dengue`
WHERE
    (classificacao_final IN ('11', '12') OR evolucao_caso = '2')
    AND evolucao_caso IS NOT NULL
    AND classificacao_final IS NOT NULL
    AND data_primeiros_sintomas IS NOT NULL
    AND data_notificacao IS NOT NULL

Running query in 'bigquery://trabalho-pratico-ml'

304139 rows affected.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304139 entries, 0 to 304138
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   sigla_uf_residencia           304095 non-null  object
 1   idade_paciente                301167 non-null  object
 2   sexo_paciente                 304126 non-null  object
 3   raca_cor_paciente             297742 non-null  object
 4   gestante_paciente             301130 non-null  object
 5   possui_doenca_autoimune       261646 non-null  object
 6   possui_diabetes               261646 non-null  object
 7   possui_doencas_hematologicas  261646 non-null  object
 8   possui_hepatopatias           261646 non-null  object
 9   possui_doenca_renal           261646 non-null  object
 10  possui_hipertensao            261646 non-null  object
 11  possui_doenca_acido_peptica   261646 non-null  object
 12  apresenta_febre               264515 non-null  object
 13 

In [20]:
save_parquet_in_chunks(df, "data/2_silver/2000-2025_alarm_severe")

Salvando arquivos Parquet:   0%|          | 0/304139 [00:00<?, ?linhas/s]

✔️ 2000-2025_alarm_severe_part_1.parquet salvo com 1.65 MB (304139 linhas)
✅ Todos os arquivos salvos com sucesso.
